In [1]:
# Morgan Fraser & Andrew Kirwan - 5/11/24
# Requires paramiko, which installs through conda:
# conda install paramiko

In [115]:
from datetime import datetime
import csv 
import os
import time
import paramiko
import pandas as pd
from event_logger import logging

In [116]:
logger = logging.getLogger()

In [117]:
router_ip = "cubesatsim4.local"
port_number = 22
router_username = "pi"
router_password = "raspberry"
script_path = "/home/pi/CubeSatSim/telem"
csv_file = "amsat_sensor_data.csv"

In [118]:
csv_file_2 = "amsat_data_turntable.csv"

In [119]:
csv_file_3 = "amsat_data_shake_test.csv"

In [120]:
csv_file_4 = 'amsat_data_turntable_accel.csv'

In [121]:
def ssh_connect():
    """ wrapper to create an ssh instance """
    ssh = paramiko.SSHClient()

    # load the host keys 
    ssh.load_system_host_keys()

    # add ssh host key if needed
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

    # connect to the amsat
    ssh.connect(hostname=router_ip,
            port=port_number,
            username=router_username,
            password=router_password,
            look_for_keys=False)
    print('Connected!')
    return ssh

In [122]:
def run_script(ssh):
    """
    Use the ssh instance to run the script and get the output 
    
    Note the timestamp is estimated on the client side (i.e. your computer)
    which may not be an accurate representation of the execution time on
    the server (i.e. the AmSat). There may be several sources of delay between
    the time Jupyter sends a command and the output is received. You should find
    a better way!
    """

    ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command(f"bash -l -c '{script_path}'")
    output = ssh_stdout.readlines()

    # get a client-side timestamp
    timestamp = datetime.isoformat(datetime.now())
    return timestamp, output

def parse_output(timestamp, output):
    """ 
    The output from the `telem` script needs tidying up. Collect it, parse it,
    and return a nicer list
    """

    data = []

    for line in output:
        print(line)
        # each line we want has a voltage and pipe ("|") divider,
        # so skip lines that don't have this
        if not line.strip() or "V" not in line or "|" not in line:
            continue

        parts = line.split("|")
        if len(parts) < 2:
            continue

        # get the parameter (+X, -X, etc)
        parameter = parts[0].strip()
        voltage_current_parts = parts[1].strip().split()

        if len(voltage_current_parts) < 3:
            continue
        voltage = float(voltage_current_parts[0])
        current = float(voltage_current_parts[2])
        data.append([timestamp, parameter, voltage, current])

        return data

In [123]:
def update_csv(data):
    """ if there is a csv file, append to it; otherwise write a new one """

    with open(csv_file_4, 'a') as file:
        writer = csv.writer(file)

        # If the file doesn't exist, write the column names
        if not os.path.exists(csv_file_4):
            writer.writerow(['Timestamp', 'Axis', 'Voltage (V)', 'Current (mA)'])

        writer.writerows(data)

    print(f"Data written to {csv_file_4}.")
        

In [124]:
ssh = ssh_connect()
try:
    print(f"Running script {script_path.split('/')[-1]} and collecting output...")
    timestamp, output = run_script(ssh)
    print(timestamp,output)
    parsed_data = parse_output(timestamp, output)
    update_csv(parsed_data)
    time.sleep(1)  # change this if you want; the number is in seconds
except KeyboardInterrupt:
    print("Script interrupted by user.")
finally:
    ssh.close()
    print("SSH connection closed.")

gaierror: [Errno 11001] getaddrinfo failed

In [ ]:
def run_cubesatsim(ssh, chunk_size=4096, read_duration=60):
    try:
        remote_command = f'bash -c -l "/home/pi/CubeSatSim/cubesatsim"'
        stdin, stdout, _ = ssh.exec_command(
            remote_command,
            get_pty=True
        )

        ssh_chunks = []
    
        start_time = time.time()
        logger.info(f"Read duration set to: {read_duration} seconds")
    
        loop_counter = 0
        while True:
            if stdout.channel.recv_ready():
                logger.debug("Stdout buffer ready to be read")
                data = stdout.channel.recv(chunk_size).decode('utf-8')
                if data:
                    ssh_chunks.append(data)
    
            if time.time() - start_time > read_duration:
                break
            time.sleep(0.1)
            loop_counter += 1

        # when the loop is done, send the ctrl+c command and close the shell
        stdin.write(chr(3))
        stdin.flush()
        time.sleep(1)
    
        next_time = time.time() - start_time
        logger.info(f"{loop_counter} loops done.")
        logger.info(f"{len(sensor_lines)} output lines read in {next_time:0.3f} seconds. Cleaning up...")

        # read whatever is left in the buffer
        while stdout.channel.recv_ready():
            data = stdout.channel.recv(chunk_size).decode('utf-8')
            if data:
                ssh_chunks.append(data)
    
        exit_status = stdout.channel.recv_exit_status()
        end_time = time.time() - start_time
        logger.info(f"Exit status received: {exit_status}")
        logger.info(f"Ending loop after {end_time:0.3f} seconds.")
    except Exception as e:
        logger.warning(f'Some error occurred: {e}')

    return ssh_chunks

In [125]:
accel = run_cubesatsim(ssh, chunk_size=4096, read_duration=60)

NameError: name 'run_cubesatsim' is not defined

In [53]:
def save_raw_to_txt(raw_chunks, filename='raw_chunks.txt'):
    """
    Save raw CubeSatSim data chunks to a text file.
    """
    with open(filename, 'w') as f:
        for chunk in raw_chunks:
            f.write(chunk)
    
    print(f"Raw data saved to {filename}")

In [54]:
save_raw_to_txt(accel)

Raw data saved to raw_chunks.txt


In [126]:
with open('raw_chunks.txt', 'r') as f:
    content = f.read()

In [127]:
def parse_cubesatsim(file):
    """
    skeleton code to help you get started
    NOTE: You must comment your code!
    """
    lines = []
    
    for line in file:
        # Clean the chunk
        stripped = line.strip()
        
        # Check if line starts with the pattern
        if stripped.startswith('Response from STEM Payload board:'):
            lines.append(stripped)
    
    return lines
    # bme_data = []
    # mpu_data = []
    # # you can use relative time rather than timestamps
    # current_time = 0

    # loop_pattern = "Loop time: 60s"
    # telemetry_pattern = "choose a pattern here"

    # for chunk in raw_chunks:
    #     # you may need to clean the stream
    #     if loop_pattern in chunk:
    #         pass

    #     if telemetry_pattern in chunk:
    #         pass
    # return bme_data, mpu_data

In [147]:
file = 'raw_chunks.txt'

In [129]:
BME280 23.30 1004.00 77.27 36.6

"\nwith open(file,'r') as f:\n    for line in f:\n        stripped = line.strip()\n        print(stripped)"

In [149]:
cubesat_data = {
    'Lat/Long' : [],
    'Loops Run' : [],
    'Loop Time' : [],
    'BME Data' : [],
    'MPU Data': [],
    'Battery Voltage' : [],
    'Battery Voltage Threshold' : [],
    'Battery Current' : [],
    'Battery Current Threshold' : [],
    'Connection Failures' : []

}

BME = {
    'Temperature' : [],
    'Pressure' : [],
    'Altitude' : [],
    'Humidity': []
}

MPU = {
    'Gyro X' : [],
    'Gyro Y' : [],
    'Gyro Z' : [],
    'Accel X' : [],
    'Accel Y' : [],
    'Accel Z' : []
}

In [152]:
def cubesat_parser(file):
    loops = 0
    fails = 0
    data_lines = []
    with open(file,'r') as f:
        for line in f:
            stripped = line.strip()
            if stripped.startswith('Response from STEM Payload'):
                data_lines.append(stripped)
        
            if 'Loop time' in line:
                loops += 1
                line_split = line.split(' ')
                cubesat_data['Loop Time'].append(float(line_split[3]))
            
            if 'Connection Failed' in line:
                fails += 1

            if len(cubesat_data['Battery Voltage']) == 0:
                if 'INFO: Battery' in line:
                    line_split = line.split(' ')
                
                
                    cubesat_data['Battery Voltage'].append(float(line_split[4]))
                    cubesat_data['Battery Voltage Threshold'].append(float(line_split[9]))
                    cubesat_data['Battery Current'].append(float(line_split[15]))
                    cubesat_data['Battery Current Threshold'].append(float(line_split[-2]))

            if 'SPARKY' in line:
                line_split = line.split(' ')
                line_split.remove(line_split[-1])
                lat,long = line_split[-2:]
                lat = float(lat)
                long = float(long)
                cubesat_data['Lat/Long'].append(lat)
                cubesat_data['Lat/Long'].append(long)

        for i in data_lines:
            if 'OK' not in i:
                i_copy = i[35:]
            else:
                i_copy = i[37:]
            strip = i_copy.split(' ')
        
            BME['Temperature'].append(float(strip[1]))
            BME['Pressure'].append(float(strip[2]))
            BME['Altitude'].append(float(strip[3]))
            BME['Humidity'].append(float(strip[4]))

            MPU['Gyro X'].append(float(strip[6]))
            MPU['Gyro Y'].append(float(strip[7]))
            MPU['Gyro Z'].append(float(strip[8]))
            MPU['Accel X'].append(float(strip[9]))
            MPU['Accel Y'].append(float(strip[10]))
            MPU['Accel Z'].append(float(strip[11]))

        
        cubesat_data['Connection Failures'].append(fails)
        cubesat_data['Loops Run'].append(loops)

In [153]:
cubesat_parser(file)

In [148]:
with open(file,'r') as f:
    for line in f:
        print(line)



SSH is enabled and the default password for the 'pi' user has not been changed.

This is a security risk - please login as the 'pi' user and type 'passwd' to set a new password.







CubeSatSim v1.2 starting...



rm: cannot remove '/home/pi/CubeSatSim/ready': No such file or directory

rm: cannot remove '/home/pi/CubeSatSim/cwready': No such file or directory

Test bus 1

I2C Bus Tested: /dev/i2c-1 

Test bus 3

I2C Bus Tested: /dev/i2c-3 

Finished testing

Mode file /home/pi/CubeSatSim/.mode contains f

Mode is FSK

Config file /home/pi/CubeSatSim/sim.cfg contains SPARKY 35 53.314098 -6.235200 

Valid latitude and longitude in config file

Lat/Long in APRS DDMM.MM format: 5318.500000/-614.059998

v1 Present with UHF BPF

Tx LED Off

Power LED On

INFO: I2C bus status 0: -1 1: 1 3: 3 camera: -1

Clearing buffer of 95 chars 

Querying payload with R to reset






























































































































In [139]:
os.getcwd()

'C:\\Users\\erisd\\Desktop\\Detector Lab\\AMSAT'

In [156]:
cubesat_data

{'Lat/Long': [53.314098, -6.2352, 53.314098, -6.2352],
 'Loops Run': [14],
 'Loop Time': [0.002,
  2.386,
  2.218,
  2.207,
  2.207,
  2.267,
  4.985,
  4.902,
  4.902,
  4.907,
  4.889,
  4.903,
  4.904,
  4.9,
  0.002,
  2.386,
  2.218,
  2.207,
  2.207,
  2.267,
  4.985,
  4.902,
  4.902,
  4.907,
  4.889,
  4.903,
  4.904,
  4.9],
 'BME Data': [],
 'MPU Data': [],
 'Battery Voltage': [3.71],
 'Battery Voltage Threshold': [3.5],
 'Battery Current': [0.1],
 'Battery Current Threshold': [100.0],
 'Connection Failures': [11]}

[3.71]

[53.314098, -6.2352]

15

In [100]:
table = {
    "time": [],
    "variable": [],
    "value1": [],
    "value2": []
}

filename = "amsat_data_turntable_accel.csv"
with open(filename) as file:
    for line in file:
        line = line.split(",")
        table["time"].append(line[0])
        table["variable"].append(line[1])
        table["value1"].append(float(line[2]))
        table["value2"].append(float(line[3]))

In [101]:
table["time seconds"] = []
for time in table["time"]:
    line = time.split(":")
    minute = int(line[1])
    second = float(line[2])
    t = (60*minute) + second
    table["time seconds"].append(t)


[0.0, 0.0, 0.0]

In [93]:
filtered_table = {
    "time": [],
    "variable": [],
    "value1": [],
    "value2": []
}

for i, val in enumerate(table["variable"]):
    if val in ['+Y', '-Y']:
        filtered_table["time"].append(table["time"][i])
        filtered_table["variable"].append(table["variable"][i])
        filtered_table["value1"].append(table["value1"][i])
        filtered_table["value2"].append(table["value2"][i])
       
        

In [14]:
#filtered_table['variable']